In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Creation of the dataframe and Dropping the useless features
to_open = "Corr-20240323_TheBlaze.csv"
output_name = to_open.split('-')[1]
date = output_name.split('_')[0][:4] + "-" + output_name.split('_')[0][4:6] + "-" + output_name.split('_')[0][6:8]
print(date)
df = pd.read_csv(f"weekly_corrected//{to_open}")
df

2024-03-23


,Unnamed: 0,time,number,latitude,longitude,cape,sp,tcw,sshf,slhf,...,skt,cin,sm,st,sd,sf,tcc,tp,mx2t6,mn2t6
0,0,2024-03-23 00:00:00,1,50.75,4.25,92.602890,100438.914,15.315906,31.0,0.0,...,280.45640,-1.317139,405.07420,282.60560,0.000000,0.000000,99.810700,0.000000,NaN,NaN
1,1,2024-03-23 00:00:00,2,50.75,4.25,24.945007,100405.164,15.219049,31.0,0.0,...,280.23830,-2.538719,399.31480,282.61124,0.000000,0.000000,99.876310,0.000000,NaN,NaN
2,2,2024-03-23 00:00:00,3,50.75,4.25,40.512480,100424.910,15.709969,31.0,0.0,...,280.39767,-2.283096,395.86136,282.55685,0.000000,0.000000,99.999916,0.000000,NaN,NaN
3,3,2024-03-23 00:00:00,4,50.75,4.25,52.306335,100458.210,15.881811,31.0,0.0,...,279.85070,NaN,399.97900,282.74564,0.000000,0.000000,99.999916,0.000000,NaN,NaN
4,4,2024-03-23 00:00:00,5,50.75,4.25,40.543335,100399.760,15.383862,31.0,0.0,...,280.31418,-0.611717,402.21655,282.64645,0.000000,0.000000,99.917510,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,1445,2024-03-30 00:00:00,46,50.75,4.25,3.424011,100175.555,19.204992,-436027.0,-39085390.0,...,283.37980,NaN,392.16962,284.36750,0.000000,2.021224,93.796844,24.643616,286.62387,284.15695
1446,1446,2024-03-30 00:00:00,47,50.75,4.25,0.976807,98940.055,21.361992,-1225285.2,-38499736.0,...,282.07578,NaN,404.78732,283.14584,0.000000,0.288708,99.999916,24.346758,286.54330,282.70410
1447,1447,2024-03-30 00:00:00,48,50.75,4.25,11.259186,99351.414,22.810540,-1621084.6,-36194420.0,...,284.15430,-2.214272,397.41043,283.59192,0.000000,0.427935,99.999916,19.314060,286.50726,285.17435
1448,1448,2024-03-30 00:00:00,49,50.75,4.25,0.000000,99154.780,11.173354,154278.0,-32553348.0,...,279.51230,NaN,370.29270,282.28775,0.000000,0.766020,23.458748,11.206761,284.92056,281.22946


In [3]:
df=df[['time','number','t2m']]

In [4]:
df

,time,number,t2m
0,2024-03-23 00:00:00,1,282.268816
1,2024-03-23 00:00:00,2,282.249441
2,2024-03-23 00:00:00,3,282.340158
3,2024-03-23 00:00:00,4,281.768189
4,2024-03-23 00:00:00,5,282.009883
...,...,...,...
1445,2024-03-30 00:00:00,46,285.451961
1446,2024-03-30 00:00:00,47,284.065358
1447,2024-03-30 00:00:00,48,286.549809
1448,2024-03-30 00:00:00,49,282.585853


In [5]:
df['t2m']=df['t2m']-273.15

/var/folders/lw/bnynzkxj0cq2wnbmqw0rvyzh0000gn/T/ipykernel_81416/836502028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['t2m']=df['t2m']-273.15


In [6]:
# Creation of the results Dataframes which will be made into one df later on
result_t2m  = pd.DataFrame(columns=['forecast_date','target','horizon','q0.025','q0.25','q0.5','q0.75','q0.975'])

In [7]:
times=df.time.unique()
number_of_hours=0
#defined earlier
#date = "2023-03-15"
row_index=0
for t in times:
  df_=df[df['time']==t]
  t2m = []
  t2m=df_.t2m.values


  horizon = str(number_of_hours)+' hour'
  result_t2m.loc[row_index]  = [date, 't2m', horizon , np.quantile(t2m,0.025),np.quantile(t2m,0.25),np.quantile(t2m,0.5),np.quantile(t2m,0.75),np.quantile(t2m,0.975)]
  row_index=row_index+1
  number_of_hours = number_of_hours + 6


In [8]:
# Dataframes concatination and result exportation
result_t2m = pd.DataFrame.to_csv(result_t2m,f"weekly_corrected/{output_name}", index=False)

In [11]:
result_t2m